# 🚀 Fine-Tune Qwen2.5-Coder for Linux Kernel Code Review

This notebook fine-tunes a code reviewer model using your Linux kernel training data.

## Steps:
1. Install libraries
2. Mount Google Drive (where your data is)
3. Train the model
4. Export to GGUF format for Ollama
5. Download the model

**Make sure to select GPU runtime**: Runtime → Change runtime type → T4 GPU

In [ ]:
# Step 1: Install Unsloth and dependencies
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" -q
!pip install transformers datasets trl accelerate peft bitsandbytes -q
print("✅ Libraries installed!")

In [ ]:
# Step 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive mounted!")

In [ ]:
# Step 3: Verify your data file exists
import os

# UPDATE THIS PATH if your file is in a different location
DATA_FILE = "/content/drive/MyDrive/premium_reasoning.jsonl"

if os.path.exists(DATA_FILE):
    size_mb = os.path.getsize(DATA_FILE) / 1_000_000
    print(f"✅ Found data file: {DATA_FILE}")
    print(f"   Size: {size_mb:.2f} MB")
else:
    print(f"❌ File not found: {DATA_FILE}")
    print("   Please upload premium_reasoning.jsonl to your Google Drive root folder")
    print("   Or update the DATA_FILE path above")

In [ ]:
# Step 4: Load the model
from unsloth import FastLanguageModel
import torch

print("Loading Qwen2.5-Coder-7B...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen2.5-Coder-7B",
    max_seq_length=4096,
    load_in_4bit=True,
    dtype=None,
)
print("✅ Model loaded!")

In [ ]:
# Step 5: Apply LoRA for efficient fine-tuning
print("Applying LoRA adapters...")
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    bias="none",
    use_gradient_checkpointing="unsloth",
)
print("✅ LoRA applied!")

In [ ]:
# Step 6: Load and format training data
from datasets import load_dataset

print("Loading dataset...")
dataset = load_dataset("json", data_files=DATA_FILE)

def format_instruction(example):
    """Format data for instruction tuning."""
    return {
        "text": f"""### Instruction
{example['instruction']}

### Input
{example['input']}

### Response
{example['output']}"""
    }

dataset = dataset["train"].map(format_instruction)
print(f"✅ Dataset loaded: {len(dataset)} examples")

In [ ]:
# Step 7: Configure training
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=4096,
    args=TrainingArguments(
        output_dir="/content/kernel_reviewer",
        num_train_epochs=1,  # Increase for better results
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        save_steps=500,
        warmup_steps=50,
        optim="adamw_8bit",
        seed=42,
    ),
)
print("✅ Trainer configured!")

In [ ]:
# Step 8: TRAIN! 🚀
print("="*50)
print("🚀 STARTING TRAINING")
print("="*50)
print(f"Dataset size: {len(dataset)} examples")
print(f"Epochs: 1")
print(f"Estimated time: ~30-60 minutes on T4 GPU")
print("="*50)

trainer_stats = trainer.train()

print("\n" + "="*50)
print("✅ TRAINING COMPLETE!")
print("="*50)

In [ ]:
# Step 9: Save the fine-tuned model
print("Saving model...")
model.save_pretrained("/content/kernel_reviewer")
tokenizer.save_pretrained("/content/kernel_reviewer")
print("✅ Model saved!")

In [ ]:
# Step 10: Export to GGUF format for Ollama
print("Exporting to GGUF format (for Ollama)...")
print("This may take 10-15 minutes...")

model.save_pretrained_gguf(
    "/content/kernel_reviewer_gguf",
    tokenizer,
    quantization_method="q4_k_m"  # Good balance of quality/size
)

print("✅ GGUF export complete!")

In [ ]:
# Step 11: Copy to Google Drive for safekeeping
import shutil
import os

# Create output directory in Drive
output_dir = "/content/drive/MyDrive/kernel_reviewer_model"
os.makedirs(output_dir, exist_ok=True)

# Copy GGUF file
gguf_files = [f for f in os.listdir("/content/kernel_reviewer_gguf") if f.endswith('.gguf')]
for f in gguf_files:
    src = f"/content/kernel_reviewer_gguf/{f}"
    dst = f"{output_dir}/{f}"
    print(f"Copying {f} to Google Drive...")
    shutil.copy(src, dst)

print(f"\n✅ Model saved to Google Drive: {output_dir}")

In [ ]:
# Step 12: Download GGUF file to your computer
from google.colab import files

print("Downloading GGUF file to your computer...")
print("(This may take a few minutes depending on file size)")

# Find the GGUF file
gguf_path = "/content/kernel_reviewer_gguf"
gguf_files = [f for f in os.listdir(gguf_path) if f.endswith('.gguf')]

if gguf_files:
    gguf_file = os.path.join(gguf_path, gguf_files[0])
    files.download(gguf_file)
    print(f"\n✅ Downloaded: {gguf_files[0]}")
else:
    print("❌ No GGUF file found")

# 🎉 Done!

## Next Steps:

1. **Save the downloaded GGUF file** to a folder on your computer

2. **Create a Modelfile** in the same folder:
```
FROM ./unsloth.Q4_K_M.gguf

SYSTEM "You are a Linux kernel developer. Given a commit message describing a bug fix, feature, or improvement, and the relevant source code context, generate the appropriate kernel patch."

PARAMETER temperature 0.3
PARAMETER top_p 0.9
```

3. **Import into Ollama**:
```bash
cd /path/to/your/model/folder
ollama create kernel-reviewer -f Modelfile
```

4. **Test your model**:
```bash
ollama run kernel-reviewer
```